In [ ]:
import sqlite3
from os import path
from time import time
from html2corpus import HTML2Corpus
from html2corpus.extractors import ReadabilityExtractor, ParagraphExtractor

KW44 = ('2015-10-26', '2015-11-02')
KW45 = ('2015-11-02', '2015-11-09')
KW46 = ('2015-11-09', '2015-11-16')
KW47 = ('2015-11-16', '2015-11-23')

db_file = path.join('..', 'Crawler', 'data', 'de_news.sqlite')
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

In [ ]:
def to_file(label, corpus):
    with open(path.join('data', 'Corpus_{}.txt'.format(label)), 'w', encoding='UTF-8') as f:
        for doc in corpus:
            f.write(doc + '\n')

def get_data(kw):
    query = 'SELECT title, html FROM Articles LEFT JOIN Sources ON Articles.source = Sources.id WHERE date BETWEEN ? and ?'
    cursor.execute(query, kw)

    data = cursor.fetchmany(100)
    
    while len(data) > 0:
        for row in data:
            title, html = row
            yield html
        data = cursor.fetchmany(100)

start = time()
corpus = HTML2Corpus(get_data(KW47), extractor=ParagraphExtractor(min_len=100))
to_file('KW47_P', corpus)
print(time() - start)
start = time()
corpus = HTML2Corpus(get_data(KW47), extractor=ReadabilityExtractor(min_len=100))
to_file('KW47_R', corpus)
print(time() - start)

conn.close()